In [ ]:
!pip install streamlit pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00


In [ ]:
!rm -rf /root/.config/Google/DriveFS
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ngrok authtoken 2otHBqHbBvAq7Tu9GHaRxPI7X5E_6SB39CnV44NQxF8ahEWsZ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import streamlit as st
import torch
from torchvision.models.detection import ssd300_vgg16
from torchvision import transforms
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# Set up Streamlit page configuration
st.set_page_config(page_title="Concrete Defect Detector", layout="wide")

# Define the label map
LABEL_MAP = {
    1: 'alligator crack',
    2: 'bearing',
    3: 'cavity',
    4: 'crack',
    5: 'drainage',
    6: 'efflorescence',
    7: 'expansion joint',
    8: 'exposed rebars',
    9: 'graffiti',
    10: 'hollowareas',
    11: 'joint tape',
    12: 'protective equipment',
    13: 'restformwork',
    14: 'rockpocket',
    15: 'rust',
    16: 'spalling',
    17: 'washouts/concrete corrosion',
    18: 'weathering',
    19: 'wetspot'
}

# Load the trained SSD model
@st.cache(allow_output_mutation=True)
def load_model(checkpoint_path):
    model = ssd300_vgg16(weights=None)
    checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    checkpoint_state_dict = checkpoint['state_dict'] if 'state_dict' in checkpoint else checkpoint

    # Filter and load state dict into the model
    filtered_state_dict = {k: v for k, v in checkpoint_state_dict.items() if k in model.state_dict() and v.shape == model.state_dict()[k].shape}
    model_state_dict = model.state_dict()
    model_state_dict.update(filtered_state_dict)
    model.load_state_dict(model_state_dict)
    model.eval()
    return model

# Load model
model_path = st.text_input("Enter the path to your model checkpoint:", "/content/drive/MyDrive/Colab Notebooks/ssd_model_trained_Concrete_FF_F_f_f_Zz.pth")
model = load_model(model_path)

# Image transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Function to visualize predictions
def visualize_predictions(image, boxes, labels, scores, threshold=0.04):
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)

    for i in range(len(boxes)):
        if scores[i] > threshold:
            box = boxes[i]
            label_idx = labels[i].item()
            label = LABEL_MAP.get(label_idx, 'Unknown')
            score = scores[i]
            rect = patches.Rectangle(
                (box[0], box[1]), box[2] - box[0], box[3] - box[1],
                linewidth=2, edgecolor='r', facecolor='none'
            )
            ax.add_patch(rect)
            ax.text(box[0], box[1], f'{label}: {score:.2f}', color='yellow', fontsize=10)
    ax.axis('off')
    st.pyplot(fig)

# Upload image
uploaded_file = st.file_uploader("Choose an image file", type=["jpg", "jpeg", "png"])
if uploaded_file is not None:
    image = Image.open(uploaded_file).convert('RGB')
    st.image(image, caption="Uploaded Image", use_column_width=True)

    # Apply transformations
    image_tensor = transform(image).unsqueeze(0)

    # Make predictions
    with torch.no_grad():
        predictions = model(image_tensor)

    # Extract predictions
    boxes = predictions[0]['boxes'].cpu().numpy() if 'boxes' in predictions[0] else []
    labels = predictions[0]['labels'].cpu().numpy() if 'labels' in predictions[0] else []
    scores = predictions[0]['scores'].cpu().numpy() if 'scores' in predictions[0] else []

    # Set threshold
    threshold = st.slider("Confidence Threshold", 0.01, 1.0, 0.04)

    # Display prediction statistics
    st.write(f"Total Predictions: {len(boxes)}")
    st.write(f"Predictions Above Threshold ({threshold}): {sum(score > threshold for score in scores)}")

    # Visualize predictions
    visualize_predictions(np.array(image), boxes, labels, scores, threshold=threshold)


2024-11-27 18:59:30.349 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-27 18:59:30.357 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-27 18:59:30.362 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-27 18:59:30.368 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching
logic used by `st.cache_data` and `st.cache_resource`. This might lead to some problems
or unexpected behavior in certain edge cases.

2024-11-27 18:59:30.382 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-2

In [ ]:
!ngrok config add-authtoken 2otHBqHbBvAq7Tu9GHaRxPI7X5E_6SB39CnV44NQxF8ahEWsZ

from pyngrok import ngrok


ngrok.kill()


public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: NgrokTunnel: "https://0e76-35-196-137-18.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py &>/dev/null&


In [ ]:
!streamlit run your_streamlit_script.py &>/content/logs.txt &


In [ ]:
!ngrok version


ngrok version 3.18.4
pyngrok version 7.2.1


In [ ]:
public_url = ngrok.connect(8501)
print(f"Public URL: {public_url}")


Public URL: NgrokTunnel: "https://92a6-35-196-137-18.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!streamlit run your_streamlit_script.py &>/content/logs.txt &


In [ ]:
public_url = ngrok.connect(8502)
print(f"Streamlit app is live at: {public_url}")


Streamlit app is live at: NgrokTunnel: "https://df84-35-196-137-18.ngrok-free.app" -> "http://localhost:8502"


In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import torch
from torchvision.models.detection import ssd300_vgg16
from torchvision import transforms
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# Define the label map
LABEL_MAP = {
    1: 'alligator crack',
    2: 'bearing',
    3: 'cavity',
    4: 'crack',
    5: 'drainage',
    6: 'efflorescence',
    7: 'expansion joint',
    8: 'exposed rebars',
    9: 'graffiti',
    10: 'hollowareas',
    11: 'joint tape',
    12: 'protective equipment',
    13: 'restformwork',
    14: 'rockpocket',
    15: 'rust',
    16: 'spalling',
    17: 'washouts/concrete corrosion',
    18: 'weathering',
    19: 'wetspot'
}

# Load the trained SSD model
model = ssd300_vgg16(weights=None)
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/ssd_model_trained_Concrete_FF_F_f_f_Zz.pth'
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
checkpoint_state_dict = checkpoint['state_dict'] if 'state_dict' in checkpoint else checkpoint

# Filter and load state dict into the model
filtered_state_dict = {k: v for k, v in checkpoint_state_dict.items() if k in model.state_dict() and v.shape == model.state_dict()[k].shape}
model_state_dict = model.state_dict()
model_state_dict.update(filtered_state_dict)
model.load_state_dict(model_state_dict)
model.eval()

# Image transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Function to visualize predictions
def visualize_predictions(image, boxes, labels, scores, threshold=0.04):
    valid_boxes = []
    valid_labels = []
    valid_scores = []

    for i in range(len(boxes)):
        if scores[i] > threshold:
            valid_boxes.append(boxes[i])
            valid_labels.append(labels[i])
            valid_scores.append(scores[i])

    if not valid_boxes:
        return None

    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)

    for i in range(len(valid_boxes)):
        box = valid_boxes[i]
        label_idx = valid_labels[i].item()
        label = LABEL_MAP.get(label_idx, 'Unknown')
        score = valid_scores[i]
        rect = patches.Rectangle(
            (box[0], box[1]), box[2] - box[0], box[3] - box[1],
            linewidth=1, edgecolor='r', facecolor='none'
        )
        ax.add_patch(rect)
        plt.text(box[0], box[1], f'{label}: {score:.2f}', color='yellow', fontsize=12)
    plt.axis('off')
    plt.tight_layout()

    return plt.gcf()

# Function to process image and make predictions
def detect_objects(image):
    # Apply transformations to the image
    image_tensor = transform(image).unsqueeze(0)

    # Make predictions
    with torch.no_grad():
        predictions = model(image_tensor)

    # Extract predictions
    boxes = predictions[0]['boxes'].cpu().numpy() if 'boxes' in predictions[0] else []
    labels = predictions[0]['labels'].cpu().numpy() if 'labels' in predictions[0] else []
    scores = predictions[0]['scores'].cpu().numpy() if 'scores' in predictions[0] else []

    # Visualize predictions
    result_image = visualize_predictions(np.array(image), boxes, labels, scores, threshold=0.04)
    return result_image

# Gradio Interface
iface = gr.Interface(
    fn=detect_objects,
    inputs=gr.Image(type="pil"),
    outputs=gr.Plot(),
    title="Concrete Structure Defect Detection",
    description="Upload an image of concrete and get defect detection results."
)

# Launch the interface
iface.launch()


<ipython-input-18-af6be239ed6a>:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2024/11/28 10:32:08 [W] [service.go:132] login to server failed: session shutdown


<IPython.core.display.Javascript object>

In [ ]:
import torch
from torchvision.models.detection import ssd300_vgg16
from torchvision import transforms
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# Define the label map
LABEL_MAP = {
    1: 'alligator crack',
    2: 'bearing',
    3: 'cavity',
    4: 'crack',
    5: 'drainage',
    6: 'efflorescence',
    7: 'expansion joint',
    8: 'exposed rebars',
    9: 'graffiti',
    10: 'hollowareas',
    11: 'joint tape',
    12: 'protective equipment',
    13: 'restformwork',
    14: 'rockpocket',
    15: 'rust',
    16: 'spalling',
    17: 'washouts/concrete corrosion',
    18: 'weathering',
    19: 'wetspot'
}

# Define solutions for each defect type
SOLUTION_MAP = {
    'alligator crack': "Solution: Use crack sealant or epoxy injections to prevent water ingress. Re-evaluate the structural integrity if cracks are extensive.",
    'bearing': "Solution: Inspect and replace if compromised, ensure proper load distribution.",
    'cavity': "Solution: Fill cavities with a cementitious or epoxy-based material to restore surface integrity.",
    'crack': "Solution: Fill small cracks with epoxy or polyurethane sealants. For larger cracks, use cement grout or epoxy injection.",
    'drainage': "Solution: Clear or repair drainage paths to prevent water accumulation on concrete surfaces.",
    'efflorescence': "Solution: Clean with a dilute acidic solution or use commercial efflorescence removers. Address moisture sources.",
    'expansion joint': "Solution: Replace damaged joints with new sealant to allow movement without cracking.",
    'exposed rebars': "Solution: Clean rust off rebars, apply a protective coating, and patch with high-strength repair mortar.",
    'graffiti': "Solution: Use graffiti removal products or apply an anti-graffiti coating to prevent future damage.",
    'hollowareas': "Solution: Inject with grout or epoxy to fill voids and restore support.",
    'joint tape': "Solution: Repair or replace joint tapes to ensure structural continuity.",
    'protective equipment': "Solution: Ensure appropriate safety gear is used and in good condition.",
    'restformwork': "Solution: Remove residual formwork and finish surfaces properly.",
    'rockpocket': "Solution: Use grout to fill rock pockets, ensuring surface continuity.",
    'rust': "Solution: Remove rust and treat rebars with anti-corrosion products. Patch concrete to prevent future exposure.",
    'spalling': "Solution: Remove loose concrete, clean exposed areas, and patch with repair mortar or epoxy.",
    'washouts/concrete corrosion': "Solution: Repair with corrosion-resistant materials, and coat surfaces for protection.",
    'weathering': "Solution: Use sealers or coatings to protect concrete from weather elements.",
    'wetspot': "Solution: Improve drainage and waterproofing to prevent water accumulation."
}

# Load the trained SSD model
model = ssd300_vgg16(weights=None)
checkpoint_path = '/content/drive/MyDrive/Colab Notebooks/ssd_model_trained_Concrete_FF_F_f_f_Zz.pth'
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
checkpoint_state_dict = checkpoint['state_dict'] if 'state_dict' in checkpoint else checkpoint

# Filter and load state dict into the model
filtered_state_dict = {k: v for k, v in checkpoint_state_dict.items() if k in model.state_dict() and v.shape == model.state_dict()[k].shape}
model_state_dict = model.state_dict()
model_state_dict.update(filtered_state_dict)
model.load_state_dict(model_state_dict)
model.eval()

# Image transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Function to visualize predictions
def visualize_predictions(image, boxes, labels, scores, threshold=0.04):
    valid_boxes = []
    valid_labels = []
    valid_scores = []

    for i in range(len(boxes)):
        if scores[i] > threshold:
            valid_boxes.append(boxes[i])
            valid_labels.append(labels[i])
            valid_scores.append(scores[i])

    if not valid_boxes:
        return None

    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)

    for i in range(len(valid_boxes)):
        box = valid_boxes[i]
        label_idx = valid_labels[i].item()
        label = LABEL_MAP.get(label_idx, 'Unknown')
        score = valid_scores[i]
        rect = patches.Rectangle(
            (box[0], box[1]), box[2] - box[0], box[3] - box[1],
            linewidth=1, edgecolor='r', facecolor='none'
        )
        ax.add_patch(rect)
        plt.text(box[0], box[1], f'{label}: {score:.2f}', color='yellow', fontsize=12)
    plt.axis('off')
    plt.tight_layout()

    return plt.gcf()

# Function to generate solutions based on detected defects
def generate_solutions(labels):
    detected_defects = set(LABEL_MAP[label.item()] for label in labels)
    solutions = {defect: SOLUTION_MAP.get(defect, "No solution available") for defect in detected_defects}
    return "\n\n".join([f"{defect}: {solution}" for defect, solution in solutions.items()])

# Function to process image and make predictions
def detect_objects(image):
    # Apply transformations to the image
    image_tensor = transform(image).unsqueeze(0)

    # Make predictions
    with torch.no_grad():
        predictions = model(image_tensor)

    # Extract predictions
    boxes = predictions[0]['boxes'].cpu().numpy() if 'boxes' in predictions[0] else []
    labels = predictions[0]['labels'].cpu().numpy() if 'labels' in predictions[0] else []
    scores = predictions[0]['scores'].cpu().numpy() if 'scores' in predictions[0] else []

    # Visualize predictions
    result_image = visualize_predictions(np.array(image), boxes, labels, scores, threshold=0.04)

    # Generate solutions based on detected labels
    solutions = generate_solutions(labels)

    return result_image, solutions

# Gradio Interface with an additional text output for solutions
iface = gr.Interface(
    fn=detect_objects,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Plot(), gr.Textbox(label="Detected Defects and Solutions")],
    title="Concrete Structure Defect Detection",
    description="Upload an image of concrete and get defect detection results along with suggested solutions."
)

# Launch the interface
iface.launch()


<ipython-input-19-e65b5fa40f87>:59: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2024/11/28 10:32:23 [W] [service.go:132] login to server failed: session shutdown


<IPython.core.display.Javascript object>

In [ ]:
import torch
from torchvision.models.detection import ssd300_vgg16
from torchvision import transforms
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# Define the label map
LABEL_MAP = {
    1: 'alligator crack',
    2: 'bearing',
    3: 'cavity',
    4: 'crack',
    5: 'drainage',
    6: 'efflorescence',
    7: 'expansion joint',
    8: 'exposed rebars',
    9: 'graffiti',
    10: 'hollowareas',
    11: 'joint tape',
    12: 'protective equipment',
    13: 'restformwork',
    14: 'rockpocket',
    15: 'rust',
    16: 'spalling',
    17: 'washouts/concrete corrosion',
    18: 'weathering',
    19: 'wetspot'
}

# Define solutions, causes, consequences for each defect type
DEFECT_INFO = {
    'alligator crack': {
        'causes': "Heavy loads, poor subgrade or base layer, aging of concrete.",
        'consequences': "Can lead to surface collapse and structural failure if not addressed.",
        'solution': "Use crack sealant or epoxy injections to prevent water ingress. Re-evaluate the structural integrity if cracks are extensive."
    },
    'bearing': {
        'causes': "Improper installation, excessive load, and lack of maintenance.",
        'consequences': "Leads to misalignment and damage to structural components.",
        'solution': "Inspect and replace if compromised, ensure proper load distribution."
    },
    'cavity': {
        'causes': "Improper concrete mixing, poor compaction, or contamination.",
        'consequences': "Weakens the structure and may lead to more severe cracks.",
        'solution': "Fill cavities with a cementitious or epoxy-based material to restore surface integrity."
    },
    'crack': {
        'causes': "Shrinkage, thermal expansion, structural loading, or poor curing.",
        'consequences': "Allows moisture ingress, which can lead to corrosion of reinforcement and further degradation.",
        'solution': "Seal cracks using epoxy or polyurethane injection. Larger cracks may require patching or resurfacing."
    },
    'drainage': {
        'causes': "Clogged or improper drainage systems, water accumulation.",
        'consequences': "Leads to water pooling, which accelerates concrete degradation.",
        'solution': "Clear drainage channels, install proper drainage systems to direct water away from the structure."
    },
    'efflorescence': {
        'causes': "Migration of salts to the surface due to moisture movement.",
        'consequences': "Leads to aesthetic issues and can indicate underlying moisture problems.",
        'solution': "Clean efflorescence with mild acid or water-based cleaners; address moisture sources to prevent recurrence."
    },
    'expansion joint': {
        'causes': "Thermal expansion and contraction of concrete.",
        'consequences': "Without expansion joints, concrete may crack due to temperature changes.",
        'solution': "Install or maintain expansion joints to allow for movement and prevent cracking."
    },
    'exposed rebars': {
        'causes': "Concrete spalling or corrosion, inadequate cover over reinforcement.",
        'consequences': "Corrosion of rebar, leading to structural weakness.",
        'solution': "Patch and repair concrete cover. Use anti-corrosion treatment for exposed rebar if necessary."
    },
    'graffiti': {
        'causes': "Unwanted painting or marking on surfaces.",
        'consequences': "Aesthetic issue, can reduce property value.",
        'solution': "Use graffiti removal solvents or pressure washing. Apply anti-graffiti coatings to prevent future markings."
    },
    'hollowareas': {
        'causes': "Poor concrete compaction, voids formed during curing.",
        'consequences': "Weakens the concrete structure and may lead to cracks.",
        'solution': "Inject grout or epoxy to fill hollow areas and restore strength."
    },
    'joint tape': {
        'causes': "Use of tape for joint sealing, may deteriorate over time.",
        'consequences': "Failure of joints, allowing moisture ingress.",
        'solution': "Replace or repair joint tape; consider a more durable sealant if necessary."
    },
    'protective equipment': {
        'causes': "Lack of protective layers or coatings.",
        'consequences': "Increased susceptibility to environmental damage.",
        'solution': "Apply protective coatings to extend concrete life."
    },
    'restformwork': {
        'causes': "Improper removal of formwork or inadequate curing.",
        'consequences': "Weakens surface, leading to potential structural issues.",
        'solution': "Repair surface with appropriate patch materials; ensure proper curing methods are used."
    },
    'rockpocket': {
        'causes': "Large aggregate concentration without enough cement paste.",
        'consequences': "Weakens structure and causes aesthetic issues.",
        'solution': "Fill rock pockets with epoxy or cementitious grout to restore structural integrity."
    },
    'rust': {
        'causes': "Exposure to moisture and air, leading to oxidation.",
        'consequences': "Corrosion of reinforcement, which can weaken structure.",
        'solution': "Remove rust and apply anti-corrosion coating. Ensure adequate cover over reinforcement."
    },
    'spalling': {
        'causes': "Freeze-thaw cycles, impact, or corrosion of reinforcement.",
        'consequences': "Weakens concrete surface and exposes reinforcement.",
        'solution': "Repair spalled areas with patching materials, address underlying causes to prevent recurrence."
    },
    'washouts/concrete corrosion': {
        'causes': "Water erosion, chemical attack on concrete.",
        'consequences': "Erosion of surface, loss of material strength.",
        'solution': "Apply corrosion-resistant coatings and protect areas from water exposure."
    },
    'weathering': {
        'causes': "Exposure to elements over time, leading to surface degradation.",
        'consequences': "Reduces concrete durability and structural integrity.",
        'solution': "Apply weather-resistant sealants and coatings to reduce weathering effects."
    },
    'wetspot': {
        'causes': "Water infiltration due to poor drainage or leaks.",
        'consequences': "Increases the risk of freeze-thaw damage and corrosion.",
        'solution': "Identify and repair sources of water infiltration. Improve drainage around the structure."
    }
}

# (Remaining code remains the same as previously provided, including the detection functions and Gradio setup.)

# Function to search for defect information based on user input
def search_defect_info(defect_type):
    info = DEFECT_INFO.get(defect_type.lower())
    if info:
        causes = info.get('causes', "No information on causes available.")
        consequences = info.get('consequences', "No information on consequences available.")
        solution = info.get('solution', "No solution available.")
        return f"**Defect Type**: {defect_type.capitalize()}\n\n**Causes**: {causes}\n\n**Long-Term Consequences**: {consequences}\n\n**Remedies**: {solution}"
    else:
        return "No information available for this defect type. Please enter a valid defect type."

# Gradio Interface with an additional search box for defect information
iface = gr.Interface(
    fn=detect_objects,
    inputs=gr.Image(type="pil"),
    outputs=[gr.Plot(), gr.Textbox(label="Detected Defects and Solutions")],
    title="Concrete Structure Defect Detection",
    description="Upload an image of concrete and get defect detection results along with suggested solutions."
)

# Gradio Interface for search
search_interface = gr.Interface(
    fn=search_defect_info,
    inputs=gr.Textbox(label="Search for Defect Information", placeholder="Enter defect type, e.g., 'crack'"),
    outputs=gr.Textbox(label="Defect Information"),
    title="Concrete Structure Defect Information Search",
    description="Enter a defect type to get information about its causes, long-term consequences, and remedies."
)

# Combine both interfaces
gr.TabbedInterface([iface, search_interface], ["Defect Detection", "Defect Search"]).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2024/11/28 10:33:01 [W] [service.go:132] login to server failed: tls: failed to verify certificate: x509: certificate has expired or is not yet valid: current time 2024-11-28T10:33:01Z is after 2024-11-28T06:24:31Z


<IPython.core.display.Javascript object>

In [22]:
import torch
from torchvision.models.detection import ssd300_vgg16
from torchvision import transforms
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import io
import base64

# Define the label map
LABEL_MAP = {
    1: 'alligator crack',
    2: 'bearing',
    3: 'cavity',
    4: 'crack',
    5: 'drainage',
    6: 'efflorescence',
    7: 'expansion joint',
    8: 'exposed rebars',
    9: 'graffiti',
    10: 'hollowareas',
    11: 'joint tape',
    12: 'protective equipment',
    13: 'restformwork',
    14: 'rockpocket',
    15: 'rust',
    16: 'spalling',
    17: 'washouts/concrete corrosion',
    18: 'weathering',
    19: 'wetspot'
}

# Define solutions, causes, consequences for each defect type
DEFECT_INFO = {
    'alligator crack': {
        'causes': "Heavy loads, poor subgrade or base layer, aging of concrete.",
        'consequences': "Can lead to surface collapse and structural failure if not addressed.",
        'solution': "Use crack sealant or epoxy injections to prevent water ingress. Re-evaluate the structural integrity if cracks are extensive."
    },
    'bearing': {
        'causes': "Improper installation, excessive load, and lack of maintenance.",
        'consequences': "Leads to misalignment and damage to structural components.",
        'solution': "Inspect and replace if compromised, ensure proper load distribution."
    },
    'cavity': {
        'causes': "Improper concrete mixing, poor compaction, or contamination.",
        'consequences': "Weakens the structure and may lead to more severe cracks.",
        'solution': "Fill cavities with a cementitious or epoxy-based material to restore surface integrity."
    },
    'crack': {
        'causes': "Shrinkage, thermal expansion, structural loading, or poor curing.",
        'consequences': "Allows moisture ingress, which can lead to corrosion of reinforcement and further degradation.",
        'solution': "Seal cracks using epoxy or polyurethane injection. Larger cracks may require patching or resurfacing."
    },
    'drainage': {
        'causes': "Clogged or improper drainage systems, water accumulation.",
        'consequences': "Leads to water pooling, which accelerates concrete degradation.",
        'solution': "Clear drainage channels, install proper drainage systems to direct water away from the structure."
    },
    'efflorescence': {
        'causes': "Migration of salts to the surface due to moisture movement.",
        'consequences': "Leads to aesthetic issues and can indicate underlying moisture problems.",
        'solution': "Clean efflorescence with mild acid or water-based cleaners; address moisture sources to prevent recurrence."
    },
    'expansion joint': {
        'causes': "Thermal expansion and contraction of concrete.",
        'consequences': "Without expansion joints, concrete may crack due to temperature changes.",
        'solution': "Install or maintain expansion joints to allow for movement and prevent cracking."
    },
    'exposed rebars': {
        'causes': "Concrete spalling or corrosion, inadequate cover over reinforcement.",
        'consequences': "Corrosion of rebar, leading to structural weakness.",
        'solution': "Patch and repair concrete cover. Use anti-corrosion treatment for exposed rebar if necessary."
    },
    'graffiti': {
        'causes': "Unwanted painting or marking on surfaces.",
        'consequences': "Aesthetic issue, can reduce property value.",
        'solution': "Use graffiti removal solvents or pressure washing. Apply anti-graffiti coatings to prevent future markings."
    },
    'hollowareas': {
        'causes': "Poor concrete compaction, voids formed during curing.",
        'consequences': "Weakens the concrete structure and may lead to cracks.",
        'solution': "Inject grout or epoxy to fill hollow areas and restore strength."
    },
    'joint tape': {
        'causes': "Use of tape for joint sealing, may deteriorate over time.",
        'consequences': "Failure of joints, allowing moisture ingress.",
        'solution': "Replace or repair joint tape; consider a more durable sealant if necessary."
    },
    'protective equipment': {
        'causes': "Lack of protective layers or coatings.",
        'consequences': "Increased susceptibility to environmental damage.",
        'solution': "Apply protective coatings to extend concrete life."
    },
    'restformwork': {
        'causes': "Improper removal of formwork or inadequate curing.",
        'consequences': "Weakens surface, leading to potential structural issues.",
        'solution': "Repair surface with appropriate patch materials; ensure proper curing methods are used."
    },
    'rockpocket': {
        'causes': "Large aggregate concentration without enough cement paste.",
        'consequences': "Weakens structure and causes aesthetic issues.",
        'solution': "Fill rock pockets with epoxy or cementitious grout to restore structural integrity."
    },
    'rust': {
        'causes': "Exposure to moisture and air, leading to oxidation.",
        'consequences': "Corrosion of reinforcement, which can weaken structure.",
        'solution': "Remove rust and apply anti-corrosion coating. Ensure adequate cover over reinforcement."
    },
    'spalling': {
        'causes': "Freeze-thaw cycles, impact, or corrosion of reinforcement.",
        'consequences': "Weakens concrete surface and exposes reinforcement.",
        'solution': "Repair spalled areas with patching materials, address underlying causes to prevent recurrence."
    },
    'washouts/concrete corrosion': {
        'causes': "Water erosion, chemical attack on concrete.",
        'consequences': "Erosion of surface, loss of material strength.",
        'solution': "Apply corrosion-resistant coatings and protect areas from water exposure."
    },
    'weathering': {
        'causes': "Exposure to elements over time, leading to surface degradation.",
        'consequences': "Reduces concrete durability and structural integrity.",
        'solution': "Apply weather-resistant sealants and coatings to reduce weathering effects."
    },
    'wetspot': {
        'causes': "Water infiltration due to poor drainage or leaks.",
        'consequences': "Increases the risk of freeze-thaw damage and corrosion.",
        'solution': "Identify and repair sources of water infiltration. Improve drainage around the structure."
    }
}


# Load the trained SSD model
model = ssd300_vgg16(weights=None)
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/ssd_model_trained_Concrete_FF_F_f_f_Zz.pth"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint)
model.eval()

# Image transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Define color palette for bounding boxes
COLOR_PALETTE = plt.get_cmap("tab20")


def get_color(label_idx):
    """Get unique color for each label."""
    return COLOR_PALETTE(label_idx / len(LABEL_MAP))


def detect_objects(image):
    if image is None:
        return None, "No image provided."

    # Transform the input image
    input_tensor = transform(image).unsqueeze(0)

    # Perform object detection
    with torch.no_grad():
        predictions = model(input_tensor)[0]

    # Filter predictions by threshold
    threshold = 0.01
    boxes = predictions['boxes']
    labels = predictions['labels']
    scores = predictions['scores']

    valid_indices = scores > threshold
    boxes = boxes[valid_indices].cpu().numpy()
    labels = labels[valid_indices].cpu().numpy()
    scores = scores[valid_indices].cpu().numpy()

    # Plot the image and overlay bounding boxes
    fig, ax = plt.subplots(1, figsize=(15, 10))
    ax.imshow(image)

    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box
        color = get_color(label)

        # Draw bounding box
        rect = patches.Rectangle(
            (x1, y1), x2 - x1, y2 - y1,
            linewidth=2, edgecolor=color, facecolor="none"
        )
        ax.add_patch(rect)
        ax.text(
            x1, y1 - 5, f"{LABEL_MAP.get(label, 'Unknown')} ({score:.2f})",
            color="white", fontsize=10,
            bbox=dict(facecolor=color, alpha=0.5)
        )

    # Save the plot as an image to embed in HTML
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    encoded_image = base64.b64encode(buf.read()).decode("utf-8")
    buf.close()
    plt.close(fig)

    # Full-page image viewer
    full_page_viewer = f"""
    <div style="display: flex; flex-direction: column; align-items: center;">
        <img src="data:image/png;base64,{encoded_image}" style="width: 100vw; height: auto; margin-bottom: 20px;"/>
        <button onclick="window.history.back()" style="padding: 10px 20px; font-size: 16px; cursor: pointer;">Go Back</button>
    </div>
    """

    return full_page_viewer, f"Detected {len(boxes)} defect(s)."


def search_defect_info(defect_type):
    defect_type = defect_type.lower()
    if defect_type in DEFECT_INFO:
        info = DEFECT_INFO[defect_type]
        causes = info['causes']
        consequences = info['consequences']
        solution = info['solution']
        return f"""
        *Defect Type*: {defect_type.capitalize()}
        *Causes*: {causes}
        *Consequences*: {consequences}
        *Solution*: {solution}
        """
    else:
        return "No information available for this defect type."


# Gradio interface for defect detection
detection_interface = gr.Interface(
    fn=detect_objects,
    inputs=[
        gr.Image(type="pil", label="Upload an Image")
    ],
    outputs=[
        gr.HTML(label="Detected Defects (Full-Page View)"),
        gr.Textbox(label="Detection Summary")
    ],
    title="Concrete Damage Detection",
    live=True
)

# Gradio interface for defect search
defect_search_interface = gr.Interface(
    fn=search_defect_info,
    inputs=gr.Textbox(label="Enter Defect Type"),
    outputs=gr.Textbox(label="Defect Information"),
    title="Defect Search",
)

# Combine interfaces into a tabbed layout
interface = gr.TabbedInterface(
    [detection_interface, defect_search_interface],
    tab_names=["Defect Detection", "Defect Search"]
)

# Launch the Gradio app
interface.launch()

<ipython-input-22-11e5a469d934>:138: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


2024/11/28 11:20:28 [W] [service.go:132] login to server failed: session shutdown


<IPython.core.display.Javascript object>

In [ ]:
import torch
from torchvision.models.detection import ssd300_vgg16
from torchvision import transforms
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np

# Define the label map
LABEL_MAP = {
    1: 'alligator crack',
    2: 'bearing',
    3: 'cavity',
    4: 'crack',
    5: 'drainage',
    6: 'efflorescence',
    7: 'expansion joint',
    8: 'exposed rebars',
    9: 'graffiti',
    10: 'hollowareas',
    11: 'joint tape',
    12: 'protective equipment',
    13: 'restformwork',
    14: 'rockpocket',
    15: 'rust',
    16: 'spalling',
    17: 'washouts/concrete corrosion',
    18: 'weathering',
    19: 'wetspot'
}

# Define defect information
DEFECT_INFO = {
    'crack': 'Cracks can weaken the structure. Solution: Apply epoxy injections or sealing agents.',
    'spalling': 'Spalling is caused by moisture. Solution: Remove loose concrete and repair with patching compounds.',
    'exposed rebars': 'Exposed rebars can lead to rusting. Solution: Clean rebars and apply anti-corrosion treatment.',
    # Add more defect information as needed...
}

# Load the trained SSD model
model = ssd300_vgg16(weights=None)
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/ssd_model_trained_Concrete_FF_F_f_f_Zz.pth"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint)
model.eval()

# Image transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

def detect_and_annotate(image):
    """Detect defects and return annotated image with defect information."""
    if image is None:
        return None, "Error: No image provided."

    # Transform the input image
    input_tensor = transform(image).unsqueeze(0)

    # Perform object detection
    with torch.no_grad():
        predictions = model(input_tensor)[0]

    # Filter predictions by threshold
    threshold = 0.3
    boxes = predictions['boxes']
    labels = predictions['labels']
    scores = predictions['scores']

    # Select valid detections
    valid_indices = scores > threshold
    boxes = boxes[valid_indices].cpu().numpy()
    labels = labels[valid_indices].cpu().numpy()
    scores = scores[valid_indices].cpu().numpy()

    # Create a plot for the annotated image
    fig, ax = plt.subplots(1, figsize=(10, 6))
    ax.imshow(image)
    defect_data = []

    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box
        defect_type = LABEL_MAP.get(label, 'Unknown')

        # Draw bounding box
        rect = patches.Rectangle(
            (x1, y1), x2 - x1, y2 - y1,
            linewidth=2, edgecolor='red', facecolor='none'
        )
        ax.add_patch(rect)

        # Annotate defect
        ax.text(
            x1, y1 - 10, f"{defect_type} ({score:.2f})",
            color='white', fontsize=10,
            bbox=dict(facecolor='red', alpha=0.5)
        )
        # Collect defect data for interactive output
        defect_data.append({
            "label": defect_type,
            "box": (x1, y1, x2, y2),
            "info": DEFECT_INFO.get(defect_type, "No additional info available.")
        })

    # Save the annotated image
    plt.axis("off")
    return fig, defect_data

def interactive_click(image, click_event):
    """Handle click events on the image and return defect info."""
    if image is None:
        return "Error: No image provided."

    # Detect defects and get their bounding boxes
    _, defect_data = detect_and_annotate(image)

    if not defect_data:
        return "No defects detected."

    # Check if the click occurred within any defect box
    x, y = click_event["x"], click_event["y"]
    for defect in defect_data:
        x1, y1, x2, y2 = defect["box"]
        if x1 <= x <= x2 and y1 <= y <= y2:
            return f"Defect: {defect['label']}\n\nDetails: {defect['info']}"

    return "No defect clicked. Please click on a defect."

# Gradio interface
image_input = gr.Image(type="pil", label="Upload an Image")
image_output = gr.Plot(label="Annotated Image")
defect_info_output = gr.Textbox(label="Defect Information", lines=10)

with gr.Blocks() as demo:
    with gr.Row():
        image_input_widget = image_input
        annotate_button = gr.Button("Detect Defects")
    with gr.Row():
        interactive_plot = image_output
        defect_info_box = defect_info_output

    annotate_button.click(
        fn=detect_and_annotate,
        inputs=image_input_widget,
        outputs=[interactive_plot, defect_info_box]
    )

    interactive_plot.select(
        fn=interactive_click,
        inputs=[image_input_widget, gr.SelectData()],
        outputs=defect_info_box
    )

# Launch the Gradio app
demo.launch()


<ipython-input-37-2f7130c9d168>:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))


KeyError: "filename 'storages' not found"

In [ ]:
import torch
from torchvision.models.detection import ssd300_vgg16
from torchvision import transforms
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import io
import base64

# Define the label map
LABEL_MAP = {
    1: 'alligator crack',
    2: 'bearing',
    3: 'cavity',
    4: 'crack',
    5: 'drainage',
    6: 'efflorescence',
    7: 'expansion joint',
    8: 'exposed rebars',
    9: 'graffiti',
    10: 'hollowareas',
    11: 'joint tape',
    12: 'protective equipment',
    13: 'restformwork',
    14: 'rockpocket',
    15: 'rust',
    16: 'spalling',
    17: 'washouts/concrete corrosion',
    18: 'weathering',
    19: 'wetspot'
}

# Define solutions, causes, consequences for each defect type
DEFECT_INFO = {
    'alligator crack': {
        'causes': "Heavy loads, poor subgrade or base layer, aging of concrete.",
        'consequences': "Can lead to surface collapse and structural failure if not addressed.",
        'solution': "Use crack sealant or epoxy injections to prevent water ingress. Re-evaluate the structural integrity if cracks are extensive."
    },
    'bearing': {
        'causes': "Improper installation, excessive load, and lack of maintenance.",
        'consequences': "Leads to misalignment and damage to structural components.",
        'solution': "Inspect and replace if compromised, ensure proper load distribution."
    },
    # Add other defects similarly
}

# Load the trained SSD model
model = ssd300_vgg16(weights=None)
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/ssd_model_trained_Concrete_FF_F_f_f_Zz.pth"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint)
model.eval()

# Image transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Define color palette for bounding boxes
COLOR_PALETTE = plt.get_cmap("tab20")

def get_color(label_idx):
    """Get unique color for each label."""
    return COLOR_PALETTE(label_idx / len(LABEL_MAP))

def detect_objects(image):
    if image is None:
        return None, "No image provided."

    # Transform the input image
    input_tensor = transform(image).unsqueeze(0)

    # Perform object detection
    with torch.no_grad():
        predictions = model(input_tensor)[0]

    # Filter predictions by threshold
    threshold = 0.01
    boxes = predictions['boxes']
    labels = predictions['labels']
    scores = predictions['scores']

    valid_indices = scores > threshold
    boxes = boxes[valid_indices].cpu().numpy()
    labels = labels[valid_indices].cpu().numpy()
    scores = scores[valid_indices].cpu().numpy()

    # Plot the image and overlay bounding boxes
    fig, ax = plt.subplots(1, figsize=(15, 10))
    ax.imshow(image)

    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box
        color = get_color(label)

        # Draw bounding box
        rect = patches.Rectangle(
            (x1, y1), x2 - x1, y2 - y1,
            linewidth=2, edgecolor=color, facecolor="none"
        )
        ax.add_patch(rect)
        ax.text(
            x1, y1 - 5, f"{LABEL_MAP.get(label, 'Unknown')} ({score:.2f})",
            color="white", fontsize=10,
            bbox=dict(facecolor=color, alpha=0.5)
        )

    # Save the plot as an image to embed in HTML
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    encoded_image = base64.b64encode(buf.read()).decode("utf-8")
    buf.close()
    plt.close(fig)

    # Full-page image viewer
    full_page_viewer = f"""
    <div style="display: flex; flex-direction: column; align-items: center;">
        <img src="data:image/png;base64,{encoded_image}" style="width: 100vw; height: auto; margin-bottom: 20px;"/>
        <button onclick="window.history.back()" style="padding: 10px 20px; font-size: 16px; cursor: pointer;">Go Back</button>
    </div>
    """

    return full_page_viewer, f"Detected {len(boxes)} defect(s)."

def search_defect_info(defect_type):
    defect_type = defect_type.lower()
    if defect_type in DEFECT_INFO:
        info = DEFECT_INFO[defect_type]
        causes = info['causes']
        consequences = info['consequences']
        solution = info['solution']
        return f"""
        <div style="padding: 10px; background-color: #f2f2f2; border-radius: 5px;">
            <h3 style="color: #333;">{defect_type.capitalize()} Information</h3>
            <b>Causes:</b> {causes}<br>
            <b>Consequences:</b> {consequences}<br>
            <b>Solution:</b> {solution}
        </div>
        """
    else:
        return "No information available for this defect type."

# Gradio interface for defect detection
detection_interface = gr.Interface(
    fn=detect_objects,
    inputs=[gr.Image(type="pil", label="Upload an Image")],
    outputs=[gr.HTML(label="Detected Defects (Full-Page View)"), gr.Textbox(label="Detection Summary")],
    title="Concrete Damage Detection",
    description="Upload an image to detect defects in concrete structures and bridges. Bounding boxes will be drawn around each detected defect.",
    live=True,
    theme="compact"
)

# Gradio interface for defect search
defect_search_interface = gr.Interface(
    fn=search_defect_info,
    inputs=gr.Textbox(label="Enter Defect Type", placeholder="e.g., Alligator Crack"),
    outputs=gr.HTML(label="Defect Information"),
    title="Defect Search",
    description="Enter the defect name to learn about its causes, consequences, and possible solutions.",
    theme="compact"
)

# Combine interfaces into a tabbed layout
interface = gr.TabbedInterface(
    [detection_interface, defect_search_interface],
    tab_names=["Defect Detection", "Defect Search"]
)

# Launch the Gradio app
interface.launch()


<ipython-input-7-3dc45694e896>:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
/usr/local/lib/py

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fa99f0d06e8e691892.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import torch
from torchvision.models.detection import ssd300_vgg16
from torchvision import transforms
import gradio as gr
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import io
import base64

# Define the label map
LABEL_MAP = {
    1: 'alligator crack',
    2: 'bearing',
    3: 'cavity',
    4: 'crack',
    5: 'drainage',
    6: 'efflorescence',
    7: 'expansion joint',
    8: 'exposed rebars',
    9: 'graffiti',
    10: 'hollowareas',
    11: 'joint tape',
    12: 'protective equipment',
    13: 'restformwork',
    14: 'rockpocket',
    15: 'rust',
    16: 'spalling',
    17: 'washouts/concrete corrosion',
    18: 'weathering',
    19: 'wetspot'
}

# Define causes, consequences, and remedies for each defect type
DEFECT_INFO = {
    'alligator crack': {
        'causes': "Heavy loads, poor subgrade or base layer, aging of concrete.",
        'consequences': "Can lead to surface collapse and structural failure if not addressed.",
        'solution': "Use crack sealant or epoxy injections to prevent water ingress. Re-evaluate the structural integrity if cracks are extensive.",
        'remedy': "Apply epoxy or polyurethane injection for minor cracks. Replace sections of the structure for severe cracking."
    },
    'bearing': {
        'causes': "Improper installation, excessive load, and lack of maintenance.",
        'consequences': "Leads to misalignment and damage to structural components.",
        'solution': "Inspect and replace if compromised, ensure proper load distribution.",
        'remedy': "Inspect bearings regularly and replace damaged ones. Ensure correct installation during construction."
    },
    'cavity': {
        'causes': "Water intrusion, poor curing, or concrete mix issues.",
        'consequences': "Weakens the structure and can lead to moisture damage.",
        'solution': "Inject grout or use a cement-based filler to seal the cavity.",
        'remedy': "Improve water drainage and ensure correct mix proportions during construction."
    },
    'crack': {
        'causes': "Overloading, thermal expansion, moisture exposure, or foundation settling.",
        'consequences': "Can cause further deterioration and expose reinforcing steel to corrosion.",
        'solution': "Fill cracks with a suitable crack filler or repair epoxy.",
        'remedy': "Monitor crack progression and seal minor cracks to prevent water ingress."
    },
    'drainage': {
        'causes': "Poor drainage system design or blocked drainage channels.",
        'consequences': "Water accumulation leading to rusting, erosion, and weakening of structural elements.",
        'solution': "Clear blockages, redesign drainage systems if necessary, and ensure proper water flow.",
        'remedy': "Maintain and clean drainage systems regularly to avoid blockages."
    },
    'efflorescence': {
        'causes': "Presence of soluble salts in the material and moisture.",
        'consequences': "Leaves visible white stains on concrete surfaces, but does not typically weaken the structure.",
        'solution': "Clean efflorescence with a diluted vinegar solution, address the source of moisture.",
        'remedy': "Improve waterproofing and ensure proper ventilation to reduce moisture ingress."
    },
    'expansion joint': {
        'causes': "Temperature changes, seismic movements, or inadequate joint design.",
        'consequences': "Can cause misalignment, cracking, or displacement of adjacent structures.",
        'solution': "Reinforce or replace the expansion joint, ensure it accommodates movement.",
        'remedy': "Inspect and maintain joints regularly to accommodate structural movements."
    },
    'exposed rebars': {
        'causes': "Corrosion of the outer concrete layer or impact damage.",
        'consequences': "Exposes the reinforcing steel to further corrosion, weakening the structure.",
        'solution': "Apply rust inhibitors and cover exposed rebars with a protective coating.",
        'remedy': "Repair the concrete cover and coat the rebars with corrosion-resistant paint."
    },
    'graffiti': {
        'causes': "Vandalism or unauthorized artwork on the surface.",
        'consequences': "Aesthetic damage, potential surface degradation from chemicals in paint.",
        'solution': "Clean with a non-abrasive cleaner and apply a protective coating.",
        'remedy': "Use anti-graffiti coatings to make surfaces easier to clean."
    },
    'hollowareas': {
        'causes': "Poor compaction during construction or soil settling.",
        'consequences': "Reduces the load-bearing capacity of the structure.",
        'solution': "Inject filler material or repair the hollow section.",
        'remedy': "Ensure proper compaction during construction to avoid future issues."
    },
    'rust': {
        'causes': "Exposure of reinforcing steel to moisture and oxygen.",
        'consequences': "Corrosion of the reinforcement, weakening the structure.",
        'solution': "Remove rust and apply protective coatings to prevent further corrosion.",
        'remedy': "Reapply protective coatings periodically and address moisture ingress."
    },
    'spalling': {
        'causes': "Corrosion of reinforcement, freeze-thaw cycles, impact damage.",
        'consequences': "Cracks and pieces of concrete breaking off, exposing rebar.",
        'solution': "Clean exposed rebar and repair the concrete with high-strength grout.",
        'remedy': "Monitor for spalling regularly and repair affected areas promptly."
    },
    'weathering': {
        'causes': "Exposure to extreme weather conditions over time.",
        'consequences': "Surface deterioration, rough texture, and eventual structural failure if not addressed.",
        'solution': "Reapply protective coatings and sealers, repair any cracks or chips.",
        'remedy': "Ensure proper waterproofing and apply protective coatings regularly."
    }
    # Add more defects similarly
}

# Load the trained SSD model
model = ssd300_vgg16(weights=None)
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/ssd_model_trained_Concrete_FF_F_f_f_Zz.pth"
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint)
model.eval()

# Image transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Define color palette for bounding boxes
COLOR_PALETTE = plt.get_cmap("tab20")

def get_color(label_idx):
    """Get unique color for each label."""
    return COLOR_PALETTE(label_idx / len(LABEL_MAP))

def detect_objects(image):
    if image is None:
        return None, "No image provided."

    # Transform the input image
    input_tensor = transform(image).unsqueeze(0)

    # Perform object detection
    with torch.no_grad():
        predictions = model(input_tensor)[0]

    # Filter predictions by threshold
    threshold = 0.01
    boxes = predictions['boxes']
    labels = predictions['labels']
    scores = predictions['scores']

    valid_indices = scores > threshold
    boxes = boxes[valid_indices].cpu().numpy()
    labels = labels[valid_indices].cpu().numpy()
    scores = scores[valid_indices].cpu().numpy()

    # Plot the image and overlay bounding boxes
    fig, ax = plt.subplots(1, figsize=(15, 10))
    ax.imshow(image)

    detected_defects = []
    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box
        defect_name = LABEL_MAP.get(label, 'Unknown')
        detected_defects.append(defect_name)

        color = get_color(label)

        # Draw bounding box
        rect = patches.Rectangle(
            (x1, y1), x2 - x1, y2 - y1,
            linewidth=2, edgecolor=color, facecolor="none"
        )
        ax.add_patch(rect)
        ax.text(
            x1, y1 - 5, f"{defect_name} ({score:.2f})",
            color="white", fontsize=10,
            bbox=dict(facecolor=color, alpha=0.5)
        )

    # Save the plot as an image to embed in HTML
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    encoded_image = base64.b64encode(buf.read()).decode("utf-8")
    buf.close()
    plt.close(fig)

    # Display remedies for detected defects
    remedy_text = ""
    for defect in detected_defects:
        if defect in DEFECT_INFO:
            remedy_text += f"<b>{defect.capitalize()}:</b><br>{DEFECT_INFO[defect]['remedy']}<br><br>"

    # Full-page image viewer
    full_page_viewer = f"""
    <div style="display: flex; flex-direction: column; align-items: center;">
        <img src="data:image/png;base64,{encoded_image}" style="width: 100vw; height: auto; margin-bottom: 20px;"/>
        <div>{remedy_text}</div>
    </div>
    """

    return full_page_viewer

# Gradio interface
# Gradio interface
iface = gr.Interface(fn=detect_objects, inputs=gr.Image(type="pil"), outputs="html")
iface.launch()



<ipython-input-11-d7ae8c0c795d>:121: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a2bbd8794a39e5fe06.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import torch
import torchvision
from torchvision import models
import gradio as gr
from PIL import Image
import numpy as np
import cv2

# Define your model structure (SSD with MobilenetV3)
model = models.detection.ssdlite320_mobilenet_v3_large(pretrained=False)

# Path to your trained model checkpoint
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/ssd_model_trained_Concrete_FF_F_f_f_Zz.pth"

# Load the checkpoint and inspect its keys
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))

# Print the checkpoint keys to identify how to load the weights
print("Checkpoint Keys:", checkpoint.keys())

# Manually load weights from the checkpoint based on the key names
try:
    # Load the model state_dict using the correct keys
    model.backbone.load_state_dict({k.replace('backbone.', ''): v for k, v in checkpoint.items() if 'backbone' in k})
    model.head.classification_head.load_state_dict({k.replace('head.', ''): v for k, v in checkpoint.items() if 'classification_head' in k})
    model.head.regression_head.load_state_dict({k.replace('head.', ''): v for k, v in checkpoint.items() if 'regression_head' in k})
except KeyError as e:
    print(f"Error loading checkpoint: {e}")

# Set the model to evaluation mode
model.eval()

# Define defect information with causes, consequences, solutions, and remedies
DEFECT_INFO = {
    'alligator crack': {
        'causes': "Heavy loads, poor subgrade or base layer, aging of concrete.",
        'consequences': "Can lead to surface collapse and structural failure if not addressed.",
        'solution': "Use crack sealant or epoxy injections to prevent water ingress. Re-evaluate the structural integrity if cracks are extensive.",
        'remedy': "Apply epoxy or polyurethane injection for minor cracks. Replace sections of the structure for severe cracking."
    },
    'bearing': {
        'causes': "Improper installation, excessive load, and lack of maintenance.",
        'consequences': "Leads to misalignment and damage to structural components.",
        'solution': "Inspect and replace if compromised, ensure proper load distribution.",
        'remedy': "Inspect bearings regularly and replace damaged ones. Ensure correct installation during construction."
    },
    'cavity': {
        'causes': "Water intrusion, poor curing, or concrete mix issues.",
        'consequences': "Weakens the structure and can lead to moisture damage.",
        'solution': "Inject grout or use a cement-based filler to seal the cavity.",
        'remedy': "Improve water drainage and ensure correct mix proportions during construction."
    },
    'crack': {
        'causes': "Overloading, thermal expansion, moisture exposure, or foundation settling.",
        'consequences': "Can cause further deterioration and expose reinforcing steel to corrosion.",
        'solution': "Fill cracks with a suitable crack filler or repair epoxy.",
        'remedy': "Monitor crack progression and seal minor cracks to prevent water ingress."
    },
    'drainage': {
        'causes': "Poor drainage system design or blocked drainage channels.",
        'consequences': "Water accumulation leading to rusting, erosion, and weakening of structural elements.",
        'solution': "Clear blockages, redesign drainage systems if necessary, and ensure proper water flow.",
        'remedy': "Maintain and clean drainage systems regularly to avoid blockages."
    },
    'efflorescence': {
        'causes': "Presence of soluble salts in the material and moisture.",
        'consequences': "Leaves visible white stains on concrete surfaces, but does not typically weaken the structure.",
        'solution': "Clean efflorescence with a diluted vinegar solution, address the source of moisture.",
        'remedy': "Improve waterproofing and ensure proper ventilation to reduce moisture ingress."
    },
    'expansion joint': {
        'causes': "Temperature changes, seismic movements, or inadequate joint design.",
        'consequences': "Can cause misalignment, cracking, or displacement of adjacent structures.",
        'solution': "Reinforce or replace the expansion joint, ensure it accommodates movement.",
        'remedy': "Inspect and maintain joints regularly to accommodate structural movements."
    },
    'exposed rebars': {
        'causes': "Corrosion of the outer concrete layer or impact damage.",
        'consequences': "Exposes the reinforcing steel to further corrosion, weakening the structure.",
        'solution': "Apply rust inhibitors and cover exposed rebars with a protective coating.",
        'remedy': "Repair the concrete cover and coat the rebars with corrosion-resistant paint."
    },
    'graffiti': {
        'causes': "Vandalism or unauthorized artwork on the surface.",
        'consequences': "Aesthetic damage, potential surface degradation from chemicals in paint.",
        'solution': "Clean with a non-abrasive cleaner and apply a protective coating.",
        'remedy': "Use anti-graffiti coatings to make surfaces easier to clean."
    },
    'hollowareas': {
        'causes': "Poor compaction during construction or soil settling.",
        'consequences': "Reduces the load-bearing capacity of the structure.",
        'solution': "Inject filler material or repair the hollow section.",
        'remedy': "Ensure proper compaction during construction to avoid future issues."
    },
    'rust': {
        'causes': "Exposure of reinforcing steel to moisture and oxygen.",
        'consequences': "Corrosion of the reinforcement, weakening the structure.",
        'solution': "Remove rust and apply protective coatings to prevent further corrosion.",
        'remedy': "Reapply protective coatings periodically and address moisture ingress."
    },
    'spalling': {
        'causes': "Corrosion of reinforcement, freeze-thaw cycles, impact damage.",
        'consequences': "Cracks and pieces of concrete breaking off, exposing rebar.",
        'solution': "Clean exposed rebar and repair the concrete with high-strength grout.",
        'remedy': "Monitor for spalling regularly and repair affected areas promptly."
    },
    'weathering': {
        'causes': "Exposure to extreme weather conditions over time.",
        'consequences': "Surface deterioration, rough texture, and eventual structural failure if not addressed.",
        'solution': "Reapply protective coatings and sealers, repair any cracks or chips.",
        'remedy': "Ensure proper waterproofing and apply protective coatings regularly."
    }
}

# Function to get remedies for a given defect
def get_remedy(defect_name):
    defect_info = DEFECT_INFO.get(defect_name.lower(), None)
    if defect_info:
        remedy = defect_info['remedy']
        return f"Remedy for {defect_name}: {remedy}"
    else:
        return "No information available for this defect."

# Preprocess the image to fit model input
def preprocess_image(image):
    # Convert image to tensor (assuming image is a PIL image)
    image = image.convert("RGB")
    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    return transform(image).unsqueeze(0)  # Add batch dimension

# Gradio Interface function to process the image and make predictions
def process_defect(input_image):
    # Preprocess the input image
    input_image = preprocess_image(input_image)

    # Run inference on the image with the model
    with torch.no_grad():
        prediction = model(input_image)  # Run the image through the model
        labels = prediction[0]['labels']
        boxes = prediction[0]['boxes']
        scores = prediction[0]['scores']

        # Assuming the model is detecting concrete defects as classes
        defect_names = ["alligator crack", "bearing", "crack", "drainage", "efflorescence", "expansion joint", "exposed rebars", "graffiti", "hollowareas", "rust", "spalling", "weathering"]

        detected_defects = []
        for i, label in enumerate(labels):
            if scores[i] > 0.5:  # Filtering by score threshold (0.5 for confidence)
                detected_defects.append(defect_names[label.item()])

        if detected_defects:
            remedies = [get_remedy(defect) for defect in detected_defects]
            return remedies
        else:
            return ["No defects detected."]

# Create a Gradio Interface
iface = gr.Interface(fn=process_defect, inputs=gr.Image(type="pil"), outputs="text")
iface.launch()


<ipython-input-21-ae19f61cc8b9>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))


Checkpoint Keys: odict_keys(['backbone.scale_weight', 'backbone.features.0.weight', 'backbone.features.0.bias', 'backbone.features.2.weight', 'backbone.features.2.bias', 'backbone.features.5.weight', 'backbone.features.5.bias', 'backbone.features.7.weight', 'backbone.features.7.bias', 'backbone.features.10.weight', 'backbone.features.10.bias', 'backbone.features.12.weight', 'backbone.features.12.bias', 'backbone.features.14.weight', 'backbone.features.14.bias', 'backbone.features.17.weight', 'backbone.features.17.bias', 'backbone.features.19.weight', 'backbone.features.19.bias', 'backbone.features.21.weight', 'backbone.features.21.bias', 'backbone.extra.0.1.weight', 'backbone.extra.0.1.bias', 'backbone.extra.0.3.weight', 'backbone.extra.0.3.bias', 'backbone.extra.0.5.weight', 'backbone.extra.0.5.bias', 'backbone.extra.0.7.1.weight', 'backbone.extra.0.7.1.bias', 'backbone.extra.0.7.3.weight', 'backbone.extra.0.7.3.bias', 'backbone.extra.1.0.weight', 'backbone.extra.1.0.bias', 'backbone.

RuntimeError: Error(s) in loading state_dict for SSDLiteFeatureExtractorMobileNet:
	Missing key(s) in state_dict: "features.0.0.0.weight", "features.0.0.1.weight", "features.0.0.1.bias", "features.0.0.1.running_mean", "features.0.0.1.running_var", "features.0.1.block.0.0.weight", "features.0.1.block.0.1.weight", "features.0.1.block.0.1.bias", "features.0.1.block.0.1.running_mean", "features.0.1.block.0.1.running_var", "features.0.1.block.1.0.weight", "features.0.1.block.1.1.weight", "features.0.1.block.1.1.bias", "features.0.1.block.1.1.running_mean", "features.0.1.block.1.1.running_var", "features.0.2.block.0.0.weight", "features.0.2.block.0.1.weight", "features.0.2.block.0.1.bias", "features.0.2.block.0.1.running_mean", "features.0.2.block.0.1.running_var", "features.0.2.block.1.0.weight", "features.0.2.block.1.1.weight", "features.0.2.block.1.1.bias", "features.0.2.block.1.1.running_mean", "features.0.2.block.1.1.running_var", "features.0.2.block.2.0.weight", "features.0.2.block.2.1

In [ ]:
def detect_objects_with_remedies(image):
    if image is None:
        return None, "No image provided."

    # Transform the input image
    input_tensor = transform(image).unsqueeze(0)

    # Perform object detection
    with torch.no_grad():
        predictions = model(input_tensor)[0]

    # Filter predictions by threshold
    threshold = 0.5
    boxes = predictions['boxes']
    labels = predictions['labels']
    scores = predictions['scores']

    valid_indices = scores > threshold
    boxes = boxes[valid_indices].cpu().numpy()
    labels = labels[valid_indices].cpu().numpy()
    scores = scores[valid_indices].cpu().numpy()

    # Prepare results for display
    remedies = []
    fig, ax = plt.subplots(1, figsize=(15, 10))
    ax.imshow(image)

    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box
        color = get_color(label)

        # Draw bounding box
        rect = patches.Rectangle(
            (x1, y1), x2 - x1, y2 - y1,
            linewidth=2, edgecolor=color, facecolor="none"
        )
        ax.add_patch(rect)
        defect_name = LABEL_MAP.get(label, 'Unknown')
        ax.text(
            x1, y1 - 5, f"{defect_name} ({score:.2f})",
            color="white", fontsize=10,
            bbox=dict(facecolor=color, alpha=0.5)
        )

        # Get remedies
        defect_info = DEFECT_INFO.get(defect_name.lower())
        if defect_info:
            remedies.append({
                "name": defect_name,
                "causes": defect_info['causes'],
                "consequences": defect_info['consequences'],
                "solution": defect_info['solution']
            })

    # Save the plot as an image to embed in HTML
    buf = io.BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    encoded_image = base64.b64encode(buf.read()).decode("utf-8")
    buf.close()
    plt.close(fig)

    # Build HTML for remedies
    remedies_html = """
    <div style="font-family: Arial, sans-serif; padding: 20px;">
        <h2>Detected Defects and Remedies</h2>
        <ul>
    """
    for remedy in remedies:
        remedies_html += f"""
        <li>
            <strong>{remedy['name']}</strong>
            <ul>
                <li><strong>Causes:</strong> {remedy['causes']}</li>
                <li><strong>Consequences:</strong> {remedy['consequences']}</li>
                <li><strong>Solution:</strong> {remedy['solution']}</li>
            </ul>
        </li>
        """
    remedies_html += "</ul></div>"

    return f"<img src='data:image/png;base64,{encoded_image}' style='width:100%;' />", remedies_html


# Update Gradio interface
detection_interface = gr.Interface(
    fn=detect_objects_with_remedies,
    inputs=[
        gr.Image(type="pil", label="Upload an Image")
    ],
    outputs=[
        gr.HTML(label="Detection Results"),
        gr.HTML(label="Remedies and Solutions")
    ],
    title="Concrete Damage Detection with Remedies",
    live=True
)

# Launch the updated interface
interface = gr.TabbedInterface(
    [detection_interface, defect_search_interface],
    tab_names=["Defect Detection", "Defect Search"]
)

interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://30569240e5e7ea4749.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
pip install pyqt5 opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.5/270.5 kB 23.3 MB/s eta 0:00:00


In [ ]:
pip install PyQt5


In [ ]:
pip install pyqt5 torch torchvision


In [ ]:
import sys
import torch
from torchvision.models.detection import ssd300_vgg16
from torchvision import transforms
from PyQt5.QtCore import Qt, QPoint
from PyQt5.QtGui import QImage, QPixmap, QPainter, QColor
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QPushButton, QLabel, QGraphicsScene, QGraphicsView, QGraphicsPixmapItem
from PIL import Image
import numpy as np

# Define the label map and defect info
LABEL_MAP = {
    1: 'alligator crack',
    2: 'bearing',
    3: 'cavity',
    4: 'crack',
    5: 'drainage',
    6: 'efflorescence',
    7: 'expansion joint',
    8: 'exposed rebars',
    9: 'graffiti',
    10: 'hollowareas',
    11: 'joint tape',
    12: 'protective equipment',
    13: 'restformwork',
    14: 'rockpocket',
    15: 'rust',
    16: 'spalling',
    17: 'washouts/concrete corrosion',
    18: 'weathering',
    19: 'wetspot'
}

DEFECT_INFO = {
    'alligator crack': "Crack due to aging or heavy load, repair with epoxy injection.",
    'bearing': "Damaged bearings need replacement.",
    'cavity': "Water intrusion causes cavities, repair with grout injection.",
    'crack': "Fill cracks with epoxy or sealant.",
    'drainage': "Ensure proper drainage to avoid further damage.",
    'efflorescence': "Clean with a vinegar solution, prevent moisture ingress.",
    'expansion joint': "Inspect and maintain joints to prevent misalignment.",
    'exposed rebars': "Reapply protective coatings to prevent rust.",
    'graffiti': "Clean with non-abrasive cleaner and apply protective coating.",
    'hollowareas': "Inject filler material or repair the hollow section.",
    'rust': "Remove rust and apply corrosion-resistant coatings."
}

# Load the SSD model
model = ssd300_vgg16(weights=None)
checkpoint_path = "/content/drive/MyDrive/Colab Notebooks/ssd_model_trained_Concrete_FF_F_f_f_Zz.pth"  # Replace with the actual model path
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'), weights_only=True)

model.load_state_dict(checkpoint)
model.eval()

# Image transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

class ImageWindow(QWidget):
    def __init__(self):
        super().__init__()

        # Initialize the window
        self.setWindowTitle('Concrete Damage Detection')
        self.setGeometry(100, 100, 800, 600)
        self.layout = QVBoxLayout()

        # Label to show the image
        self.image_label = QLabel(self)
        self.layout.addWidget(self.image_label)

        # Button to upload an image
        self.upload_button = QPushButton('Upload Image', self)
        self.upload_button.clicked.connect(self.upload_image)
        self.layout.addWidget(self.upload_button)

        self.setLayout(self.layout)

    def upload_image(self):
        # Open file dialog to upload an image
        file_path, _ = QFileDialog.getOpenFileName(self, "Open Image", "", "Images (*.png *.jpg *.bmp *.jpeg)")
        if file_path:
            self.process_image(file_path)

    def process_image(self, image_path):
        # Load the image
        image = Image.open(image_path)
        image = image.convert("RGB")

        # Transform the image
        input_tensor = transform(image).unsqueeze(0)

        # Perform object detection
        with torch.no_grad():
            predictions = model(input_tensor)[0]

        # Filter out low-confidence predictions
        threshold = 0.5
        boxes = predictions['boxes']
        labels = predictions['labels']
        scores = predictions['scores']

        valid_indices = scores > threshold
        boxes = boxes[valid_indices].cpu().numpy()
        labels = labels[valid_indices].cpu().numpy()

        # Convert the image to QPixmap and display it
        pixmap = QPixmap(image_path)
        self.image_label.setPixmap(pixmap)
        self.image_label.adjustSize()

        self.detected_boxes = []
        for box, label in zip(boxes, labels):
            x1, y1, x2, y2 = box
            defect_name = LABEL_MAP.get(label, 'Unknown')
            self.detected_boxes.append((defect_name, QRectF(x1, y1, x2 - x1, y2 - y1)))

    def paintEvent(self, event):
        if not hasattr(self, 'detected_boxes'):
            return

        painter = QPainter(self)
        for defect_name, rect in self.detected_boxes:
            painter.setPen(QColor(255, 0, 0))
            painter.setBrush(QColor(255, 0, 0, 50))
            painter.drawRect(rect)
            painter.drawText(rect.topLeft(), defect_name)

if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = ImageWindow()
    window.show()
    sys.exit(app.exec_())
